In [1]:
#HyprParams
inputdim = 10

In [1]:
#Sequential 모델은 레이어를 선형적으로 쌓는다.
from keras  import Sequential
model = Sequential()

model.add(Dense(units=32, activation='relu', inputdim=inputdim))
model.add(Dense(units=32, activation='sigmoid'))

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow
